In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE84634"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE84634"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE84634.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84634.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE84634.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of peripheral blood mononuclear cells from adults with sickle cell disease (University of Chicago cohort)"
!Series_summary	"Sickle cell disease is associated with systemic complications, many associated with either severity of disease or increased risk of mortality. We sought to identify a circulating gene expression profile whose predictive capacity spanned the spectrum of these poor outcomes in sickle cell disease."
!Series_summary	"The Training cohort consisted of patients with SCD who were prospectively recruited from the University of Illinois. The Testing cohort consisted of a combination of patients prospectively seen at two separate institutions including the University of Chicago and Howard University."
!Series_overall_design	"The gene expression of PBMC from 38 sickle cell disease patients from University of Chicago were analyzed."
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood'], 1: ['cell type: mononu

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from PBMCs
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From sample characteristics, we can see "disease: sickle cell disease" corresponds to our trait
trait_row = 2  # "disease: sickle cell disease"
age_row = None  # Age is not available in the sample characteristics
gender_row = None  # Gender is not available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary format (0 = control, 1 = case)."""
    if value is None:
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # From the data, everyone has sickle cell disease
    # So this is a constant feature with all 1s
    if "sickle cell disease" in value.lower():
        return 1
    else:
        return None  # For any unexpected values

def convert_age(value):
    """Convert age value to continuous format."""
    # Not used as age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary format (0 = female, 1 = male)."""
    # Not used as gender data is not available
    return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, clinical data is available
if trait_row is not None:
    # Assume clinical_data is already loaded from a previous step
    clinical_data = pd.DataFrame({"characteristics_ch1": ["disease: sickle cell disease"] * 38})  # Based on the description of 38 SCD patients
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'characteristics_ch1': [1.0]}
Clinical data saved to ../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE84634.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM2243346"	"GSM2243347"	"GSM2243348"	"GSM2243349"	"GSM2243350"	"GSM2243351"	"GSM2243352"	"GSM2243353"	"GSM2243354"	"GSM2243355"	"GSM2243356"	"GSM2243357"	"GSM2243358"	"GSM2243359"	"GSM2243360"	"GSM2243361"	"GSM2243362"	"GSM2243363"	"GSM2243364"	"GSM2243365"	"GSM2243366"	"GSM2243367"	"GSM2243368"	"GSM2243369"	"GSM2243370"	"GSM2243371"	"GSM2243372"	"GSM2243373"	"GSM2243374"	"GSM2243375"	"GSM2243376"	"GSM2243377"	"GSM2243378"	"GSM2243379"	"GSM2243380"	"GSM2243381"	"GSM2243382"	"GSM2243383"
First data line: 2315554	6.087148013	6.505591352	5.921904717	6.15941806	6.138992908	6.032789074	6.625662972	6.089488274	6.240933297	6.014541988	6.426659671	6.494851638	6.218330081	6.189424379	6.351294575	6.507638473	5.986528622	6.190491525	6.075310436	6.304058565	6.304614819	6.753211011	6.097934478	6.23269389	6.098436019	6.311435772	6.188093249	5.776282784	5.939137591	6.056735194	6.32142099	6.341911205	6.161502475	6.343962963	6.54086782	6.221432023	6.

### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers from the previous output
# The identifiers shown (2315554, 2315633, etc.) appear to be numeric IDs, 
# not standard human gene symbols which are typically alphanumeric (e.g. BRCA1, TP53)
# These are likely probe IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE84634
Line 6: !Series_title = Gene expression of peripheral blood mononuclear cells from adults with sickle cell disease (University of Chicago cohort)
Line 7: !Series_geo_accession = GSE84634
Line 8: !Series_status = Public on Sep 08 2017
Line 9: !Series_submission_date = Jul 20 2016
Line 10: !Series_last_update_date = Feb 18 2019
Line 11: !Series_pubmed_id = 28373264
Line 12: !Series_summary = Sickle cell disease is associated with systemic complications, many associated with either severity of disease or increased risk of mortality. We sought to identify a circulating gene expression profile whose predictive capacity spanned the spectrum of these poor outcomes in sickle cell disease.
Line 13: !Series_


Gene annotation preview:
{'ID': [2315100, 2315106, 2315109, 2315111, 2315113], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (As

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From examining the gene annotation data, we can see:
# - 'ID' column contains numeric identifiers matching those in the gene expression data
# - 'gene_assignment' column contains gene symbol information

# First, obtain the gene expression data if we haven't already
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")

# Get gene annotation data from SOFT file
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# 2. Create mapping dataframe with ID and gene symbols
# Extract the 'ID' and 'gene_assignment' columns for mapping
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print(f"First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print(f"First few rows of gene expression data:")
print(gene_data.head())

# Apply gene symbol normalization to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene expression data shape: {gene_data.shape}")

# Save gene expression data to output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data shape: (22011, 38)


Gene annotation data shape: (1153375, 12)
Mapping dataframe shape: (316481, 2)
First few rows of mapping dataframe:
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---


Mapped gene expression data shape: (48895, 38)
First few rows of gene expression data:
      GSM2243346  GSM2243347  GSM2243348  GSM2243349  GSM2243350  GSM2243351  \
Gene                                                                           
A-     15.509117   15.397530   15.808708   15.439154   15.664632   15.683596   
A-2     2.343726    2.239466    2.320720    2.421121    2.389999    2.265380   
A-52    4.393986    4.424308    4.358994    4.405801    4.404844    4.336857   
A-E     1.381988    1.407230    1.351711    1.373573    1.373060    1.288748   
A-I     4.520968    4.413596    4.748982    4.810093    4.669394    4.478377   

      GSM2243352  GSM2243353  GSM2243354  GSM2243355  ...  GSM2243374  \
Gene                                                  ...               
A-     14.417419   15.318694   15.801011   15.815661  ...   15.801735   
A-2     2.383855    2.341696    2.324946    2.263749  ...    2.363010   
A-52    4.457693    4.427117    4.329272    4.250242  ...   

Gene expression data saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84634.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains only {trait} patients with no healthy controls, making it unsuitable for case-control analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (18418, 38)
First few genes with their expression values after normalization:
          GSM2243346  GSM2243347  GSM2243348  GSM2243349  GSM2243350  \
Gene                                                                   
A1BG        1.629561    1.650178    1.623354    1.674790    1.693946   
A1BG-AS1    1.629561    1.650178    1.623354    1.674790    1.693946   
A1CF        1.021493    1.015058    0.983417    1.029269    0.977699   
A2M         2.297381    2.518467    2.445836    2.369931    2.310326   
A2ML1       1.144764    1.230670    1.232296    1.259902    1.492003   

          GSM2243351  GSM2243352  GSM2243353  GSM2243354  GSM2243355  ...  \
Gene                                                                  ...   
A1BG        1.585265    1.601531    1.629005    1.595809    1.503266  ...   
A1BG-AS1    1.585265    1.601531    1.629005    1.595809    1.503266  ...   
A1CF        0.943049    1.038536    1.008471    0.984444    0.999792  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84634.csv
Raw clinical data shape: (3, 39)
Clinical features:
                    GSM2243346  GSM2243347  GSM2243348  GSM2243349  \
Sickle_Cell_Anemia         1.0         1.0         1.0         1.0   

                    GSM2243350  GSM2243351  GSM2243352  GSM2243353  \
Sickle_Cell_Anemia         1.0         1.0         1.0         1.0   

                    GSM2243354  GSM2243355  ...  GSM2243374  GSM2243375  \
Sickle_Cell_Anemia         1.0         1.0  ...         1.0         1.0   

                    GSM2243376  GSM2243377  GSM2243378  GSM2243379  \
Sickle_Cell_Anemia         1.0         1.0         1.0         1.0   

                    GSM2243380  GSM2243381  GSM2243382  GSM2243383  
Sickle_Cell_Anemia         1.0         1.0         1.0         1.0  

[1 rows x 38 columns]
Clinical features saved to ../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE84634.csv
Linked data shape: (

Data shape after handling missing values: (38, 18419)
Quartiles for 'Sickle_Cell_Anemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Sickle_Cell_Anemia' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
